In [1]:
%run ./utility.ipynb

# Gradient Calculations

Here we compute a single gradient using 2 different methods. A gradient is usually denoted by $\nabla$ or $g$. 

## Method one: Finite Differences Methods

The Finite Differences Method are a numerical analysis technique for solving differential equations by approximating derivatves using finite differences. Using these finite differences we can approximate a gradient of a function.

So what is Finite Differences exactly? It is a mathematical expression of the form $f(x+b)-f(x+a)$.

There are three basic types that are commonly considered for this method: Forward, Backward, and Central.

Forward differences is calcuated by $\nabla = \frac{f(x+h)-f(x)}{h}$

Backward differences is calcuated by $\nabla = \frac{f(x) - f(x-h)}{h}$

Central differences is calcuated by $\nabla = \frac{f(x+\frac{h}{2})-f(x-\frac{h}{2})}{h}$

### The Function Implemented

Below we have all three methods implemented controlled by a mode flag.

For Gradient Descent, we consider $f$ to be our loss function and h to be a minute difference from a pixel's value. We iterate over every pixel and calculate the loss needed for equation used ($f(x+h)$, $f(x-h)$, or $f(x+\frac{h}{2})-f(x-\frac{h}{2})$). This gives us the gradient for each pixel.


In [ ]:
def gradient(data_list: list[data], coords, image, mode = 1): # 0 For central, -1 for backward, 1 for forward
    image_copy = np.copy(image, subok=True)
    upper_diff: float
    lower_diff: float
    h: float
    gradient_arr = np.empty(np.shape(image),dtype=np.complex_)
    if (mode == 0): # Central difference
        for row in range(len(image)):
            for col in range(len(image[row])):
                image_copy[row,col] += 1e-6 / 2
                upper_diff = loss(image_copy, data_list, coords)
                image_copy[row,col] -= 1e-6
                lower_diff = loss(image_copy, data_list, coords)
                image_copy[row,col] = image[row,col] # Reset that pixel to original value
                gradient_arr[row,col] = (upper_diff - lower_diff) / 1e-6
    elif (mode == -1): # Backward difference
        upper_diff = loss(image, data_list, coords)
        for row in range(len(image)):
            for col in range(len(image[row])):
                image_copy[row,col] -= 1e-8
                lower_diff = loss(image_copy, data_list, coords)
                gradient_arr[row,col] = (upper_diff - lower_diff) / 1e-8
                image_copy[row,col] = image[row,col]
    elif (mode == 1) : # Forward difference is default
        lower_diff = loss(image, data_list, coords)
        for row in range(len(image)):
            for col in range(len(image[row])):
                image_copy[row,col] += 1e-8 
                upper_diff = loss(image_copy, data_list, coords)
                gradient_arr[row,col] = (upper_diff - lower_diff) / 1e-8
                image_copy[row,col] = image[row,col]
    else:
        raise ValueError('Incorrect mode for finite differences')
    return gradient_arr

Here we calculate gradient using some preprocessing and derivatives. We also reduce time by using a selection approach.

In [ ]:
def preprocess_gradient(data_list: list[data], coords, image):
    r, c = np.shape(image)
    preprocessed = np.empty([r,c,len(data_list),2], dtype=np.complex_)
    for row in range(len(image)):
        for col in range(len(image[row])):
            for datum in range(len(data_list)):
                term = ((2*np.pi*1j)/image.size)*(row*data_list[datum].u + col*data_list[datum].v) #.size for numpy array returns # of rows * # of cols
                term_1 = np.exp(term)
                term_2 = np.exp(-1*term)
                preprocessed[row,col,datum,0] = term_1
                preprocessed[row,col,datum,1] = term_2
    return preprocessed

In [ ]:
def gradient_2(data_list: list[data], coords, coeffs, image, subset_percent = 10, FOV = 100*u.uas.to(u.rad)):
    gradient_arr = np.empty(np.shape(image)) # Because we are in real space
    vis_images = interpolate(image, coords, FOV)
    selection = np.random.choice(np.arange(len(data_list)), size=len(coords)*subset_percent//100, replace=False) # selection is full of indicies
    for row in range(len(image)):
        for col in range(len(image[row])):
            gradient_sum = 0
            for i in selection:
                vis_data = data_list[i].vis_data
                vis_image = vis_images[i]
                term_1 = coeffs[row,col,i,0] * (np.conj(vis_image) - np.conj(vis_data))
                term_2 = coeffs[row,col,i,1] * (vis_image - vis_data)
                gradient_sum += (term_1 + term_2)/(data_list[i].sigma ** 2)
            gradient_arr[row,col] = gradient_sum
    return gradient_arr